In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from ray.tune.schedulers import PopulationBasedTraining
from sklearn.model_selection import KFold
from ray.tune.integration.pytorch_lightning import TuneReportCallback
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score as sklearn_r2_score


In [ ]:
df=pd.read_csv('./cleaned_datasets/admission_data.csv',index_col=0)  

In [ ]:
df

In [ ]:
y = df.pop('Chance of Admit ')
X = df
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)


In [ ]:
X_train

In [ ]:
X_train_with_con = X_train.drop(columns=['CGPA'])
X_test_with_con = X_test.drop(columns=['CGPA'])

In [ ]:
X_train_with_con

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_with_con)
X_test_scaled = scaler.transform(X_test_with_con)

In [ ]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self,yhat,y):
        return torch.sqrt(self.mse(yhat,y))

In [ ]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, hidden_layers, hidden_units, dropout, dropout_array, input_dim, output_dim, activation=nn.ReLU(),norm=False):
        if hidden_layers != len(hidden_units):
            print("Error: wrong size of hidden_layers or hidden_units")
            return
        layers = []
        i = 0
        if norm:
            layers.append(nn.BatchNorm1d(input_dim))

        super(Net, self).__init__()
        
        layers.append(nn.Linear(input_dim, hidden_units[i]))
        layers.append(activation)
        if dropout:
            layers.append(nn.Dropout(dropout_array[i]))

        for _ in range(hidden_layers - 1):
            i += 1
            layers.append(nn.Linear(hidden_units[i-1], hidden_units[i]))
            layers.append(activation)
            if dropout:
                layers.append(nn.Dropout(dropout_array[i]))

        layers.append(nn.Linear(hidden_units[-1], output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.net(x)

In [ ]:
model = Net(hidden_layers=3, hidden_units=[24, 24, 24], dropout=True, dropout_array=[0.2, 0.3, 0.4], input_dim=13, output_dim=1, activation=nn.Sigmoid(),norm=True)


In [ ]:
from torchsummary import summary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
model = model.to(device)

In [ ]:
summary(model,input_size=(13,1))

In [ ]:
def create_dataloader(X_train, X_test, y_train, y_test, batch_size=32):
    # Create custom Dataset objects for train and test data
    train_dataset = DataFrameDataset(X_train, y_train)
    test_dataset = DataFrameDataset(X_test, y_test)

    # Create DataLoaders for train and test data
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader
class DataFrameDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = torch.tensor(X_data, dtype=torch.float32)
        self.y_data = torch.tensor(y_data, dtype=torch.float32)

    def __len__(self):
        return len(self.X_data)

    def __getitem__(self, idx):
        return self.X_data[idx], self.y_data[idx]

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

def cross_val_train(config, n_splits=5):
    kf = KFold(n_splits=n_splits)
    r2_scores = []
    rmse_scores = []

    for train_index, test_index in kf.split(X_train_scaled, y_train.values):
        X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
        y_train_cv, y_test_cv = y_train.values[train_index], y_train.values[test_index]

        train_loader, test_loader = create_dataloader(X_train_cv, X_test_cv, y_train_cv, y_test_cv, batch_size=config["batch_size"])

        input_dim = X_train_cv.shape[1]
        output_dim = 1
        net = Net(hidden_layers=config["hidden_layers"], hidden_units=config["hidden_units"], dropout=config["dropout"], dropout_array=config["dropout_array"], input_dim=input_dim, output_dim=output_dim, activation=config["activation"], norm=config["norm"])

        device = "cuda" if torch.cuda.is_available() else "cpu"
        net.to(device)

        criterion = RMSELoss()
        optimizer = optim.Adam(net.parameters(), lr=config["lr"])

        for epoch in range(config["epochs"]):
            running_loss = 0.0
            for i, (inputs, labels) in enumerate(train_loader):
                inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
                optimizer.zero_grad()
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            

        with torch.no_grad():
            test_loss = 0.0
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item()

            y_pred = net(torch.Tensor(X_test_cv).to(device)).cpu().numpy()
            r2_scores.append(sklearn_r2_score(y_test_cv, y_pred))
            rmse_scores.append(np.sqrt(mean_squared_error(y_test_cv, y_pred)))
            tune.report(r2_curr=sklearn_r2_score(y_test_cv, y_pred),loss_curr=np.sqrt(mean_squared_error(y_test_cv, y_pred)))
            #print(r2_scores)
            #print(rmse_scores)

    avg_r2 = np.mean(r2_scores)
    avg_rmse = np.mean(rmse_scores)
    tune.report(loss=avg_rmse,train_loss=running_loss,r2_score=avg_r2,r2_curr=r2_scores[-1])

    #return avg_rmse

#best_config = {'batch_size': 8, 'lr': 0.00035262507645898283, 'epochs': 500, 'hidden_layers': 3, 'hidden_units': [128, 128, 64], 'dropout': False, 'dropout_array': [0.39233452918726697, 0.3225943639867505, 0.6472032609256847], 'activation': nn.ReLU(), 'norm': False}

#avg_r2, avg_rmse = cross_val_train(best_config)
#print(f"Average R2 score: {avg_r2}")
#print(f"Average RMSE: {avg_rmse}")


In [ ]:
 train_ds = TensorDataset(torch.Tensor(X_train_scaled), torch.Tensor(y_train.to_numpy()))

In [ ]:
train_ds

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
device

In [ ]:
def generate_hidden_units(hidden_layers):
    return np.random.choice(a=[16,32,64,128], size=hidden_layers).tolist()

search_space = {
    "batch_size": tune.choice([8,16,32, 64, 128]),
    "lr": tune.loguniform(1e-7, 1e-2),
    "epochs": tune.choice([25,50, 100, 150,300,500]),
    #"epochs": tune.choice([25]),
    "hidden_layers": tune.randint(1, 4),
    "hidden_units": tune.sample_from(lambda spec: generate_hidden_units(spec.config.hidden_layers)),
    "dropout": tune.choice([True, False]),
    "dropout_array": tune.sample_from(lambda spec: np.random.uniform(low=0.4, high=0.6, size=spec.config.hidden_layers).tolist()),
    "activation": tune.choice([nn.ReLU()]),
    "norm": tune.choice([True, False])
}

In [ ]:
scheduler = ASHAScheduler(metric="r2_curr", mode="max", grace_period=2, reduction_factor=2)
reporter = CLIReporter(metric_columns=["loss", "training_iteration","train_loss"])

In [ ]:
analysis = tune.run(
        cross_val_train,
        resources_per_trial={"cpu": 8, "gpu": 1 if torch.cuda.is_available() else 0},
        config=search_space,
        num_samples=100,
        scheduler=scheduler,
        progress_reporter=reporter)

print("Best hyperparameters found were: ", analysis.get_best_config(metric="r2_score", mode="max"))

In [ ]:
analysis.results_df.sort_values(by=['r2_score', 'loss'],na_position='first').tail(10)

In [ ]:
best_config={'batch_size': 16, 'lr': 0.000157	, 'epochs': 500, 'hidden_layers': 1, 'hidden_units': [128], 'dropout': False, 'dropout_array': [0.5262243916801222, 0.17160622713412305, 0.28392396173952034], 'activation': nn.ReLU(), 'norm': True}

In [ ]:
best_config.update({"patience":10})

In [ ]:
best_config

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

def cross_val_train_2(config, n_splits=5):
    kf = KFold(n_splits=n_splits)
    r2_scores = []
    rmse_scores = []

    for train_index, test_index in kf.split(X_train_scaled, y_train.values):
        X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
        y_train_cv, y_test_cv = y_train.values[train_index], y_train.values[test_index]

        train_loader, test_loader = create_dataloader(X_train_cv, X_test_cv, y_train_cv, y_test_cv, batch_size=config["batch_size"])

        input_dim = X_train_cv.shape[1]
        output_dim = 1
        net = Net(hidden_layers=config["hidden_layers"], hidden_units=config["hidden_units"], dropout=config["dropout"], dropout_array=config["dropout_array"], input_dim=input_dim, output_dim=output_dim, activation=config["activation"], norm=config["norm"])

        device = "cuda" if torch.cuda.is_available() else "cpu"
        net.to(device)

        criterion = RMSELoss()
        optimizer = optim.Adam(net.parameters(), lr=config["lr"])

        for epoch in range(config["epochs"]):
            running_loss = 0.0
            for i, (inputs, labels) in enumerate(train_loader):
                inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
                optimizer.zero_grad()
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            

        with torch.no_grad():
            test_loss = 0.0
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item()

            y_pred = net(torch.Tensor(X_test_cv).to(device)).cpu().numpy()
            r2_scores.append(sklearn_r2_score(y_test_cv, y_pred))
            rmse_scores.append(np.sqrt(mean_squared_error(y_test_cv, y_pred)))
            #print(r2_scores)
            #print(rmse_scores)

    avg_r2 = np.mean(r2_scores)
    avg_rmse = np.mean(rmse_scores)

    return avg_r2, avg_rmse
avg_r2, avg_rmse = cross_val_train_2(best_config)
print(f"Average R2 score: {avg_r2}")
print(f"Average RMSE: {avg_rmse}")


In [ ]:
import numpy as np
from skopt import gp_minimize

fixed_config = {
    'hidden_layers': 1,
 'hidden_units': [128],
 'dropout': False,
 'dropout_array': [0.38283509966903684,
  0.14655095633529086,
  0.5393755103727268],
 'activation': nn.ReLU(),
 'norm': True
}

# The objective function to minimize (negative R^2)
def objective(params):
    batch_size, lr, epochs = params
    config = {'batch_size': int(batch_size), 'lr': lr, 'epochs': int(epochs)}
    config.update(fixed_config)

    r2, rmse = cross_val_train_2(config)
    return rmse  # Minimize negative R^2 to maximize R^2

# Hyperparameter search space
space = [
    (8, 256),  # batch_size
    (1e-4, 1e-2, 'log-uniform'),  # lr
    (30, 250)  # epochs
]

# Bayesian optimization using Gaussian Process
result = gp_minimize(
    func=objective,
    dimensions=space,
    n_calls=100,  # Number of iterations
    verbose=True
)

# Print the optimized hyperparameters
print("Best hyperparameters found:")
print("Batch size:", result.x[0])
print("Learning rate:", result.x[1])
print("Epochs:", result.x[2])
